In [9]:

!jupyter nbconvert --to html --TemplateExporter.exclude_code_cell=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True scc2425-lab9.ipynb 2> /dev/null
!jupyter nbconvert --to slides --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True scc2425-lab9.ipynb 2> /dev/null

# Cloud Computing Systems
## 2024/25

Lab 9
https://smduarte.github.io/scc2425/

Sérgio Duarte, Kevin Gallagher 

# Goals

+ Access control for web applications based on Cookies

# Approach to implement access control!


### Step 1 

 Users must log in to the system;
 
### Step 2

Before executing operations, check if the user is allowed to execute the operation.



# Authentication endpoint

### Step 1 - Users must log in to the system;

Endpoint: **GET /login** returns the login page, including a form to supply the **user credentials**;

Endpoint: **POST /login** verifies the users credentials, passed as **form data**. Creates a [Cookie](https://en.wikipedia.org/wiki/HTTP_cookie) to serve as proof of authentication.

### Authentication Endpoint: GET /rest/login 
```java
@Path(Authentication.PATH)
public class Authentication {
	static final String PATH = "login";
    static final String LOGIN_PAGE = "login.html";
	
	@GET
	@Produces(MediaType.TEXT_HTML)
	public String login() {
		try {
			var in = getClass().getClassLoader().getResourceAsStream(LOGIN_PAGE);
			return new String( in.readAllBytes() );			
		} catch( Exception x ) {
			throw new WebApplicationException( Status.INTERNAL_SERVER_ERROR );
		}
	}
```

Note: The GET endpoint in the authentication returns the contents of a login html page stored in the **WAR** archive as a resource.

### Authentication Endpoint: POST /rest/login 

```java
@POST
public Response login( @FormParam(USER) String user, @FormParam(PWD) String password ) {
    boolean pwdOk = ... ; // check user passwd
	if (pwdOk) {
        String uid = UUID.randomUUID().toString();
		var cookie = new NewCookie.Builder(COOKIE_KEY)
					.value(uid).path("/")
					.comment("sessionid")
					.maxAge(MAX_COOKIE_AGE)
					.secure(false)
					.httpOnly(true)
					.build();
			
		FakeRedisLayer.getInstance().putSession( new Session( uid, user));	
			
        return Response.seeOther(URI.create( REDIRECT_TO_AFTER_AUTH ))
                   .cookie(cookie) 
                   .build();
	} else
		throw new NotAuthorizedException("Incorrect login");
}
```

The POST /rest/login endpoint is the target of the form returned previously. 

The user credentials supplied in the form are encoded as *FormData* and can be captured as method parameters using **@FormParam** annotations.

If authentication succeeds, we return a [Cookie](https://en.wikipedia.org/wiki/HTTP_cookie). The cookie will be included by the browser/HTTP client in subsequent requests. It is intended to serve as proof of authentication while it is valid.

The cookie needs a key and value. Here the value is just an unique identifier. It is possible to control the how long the cookie will be valid by providing a maximum allowed age. Other flags can be used to improve security, such as only support https communications.

When the login succeeds the client is redirected automatically to a landing endpoint...

The cookie represents an authenticated user session that can be shared across all application server instances via a RedisCache. In this example, we are faking the RedisCache to provide a simple working example with a local deployment of the web app.

# Access control (1)

###  Step 2 - ... check if the user is allowed to execute the operation.


In methods that require access control, **use the cookie** to know which user is calling the method.

```java
@Path("/ctrl")
public class ControlResource
{
	@Path("/version2")
	@GET
	@Produces(MediaType.TEXT_HTML)
	public String version2( @CookieParam(Authentication.COOKIE_KEY) Cookie cookie) {
        
        var session = Authentication.validateSession(cookie, ADMIN); // will throw NOT AUTHORIZED
        // user admin authenticated
    }
    
}
```        

Note: [@CookieParam](https://jakarta.ee/specifications/platform/9/apidocs/jakarta/ws/rs/cookieparam) can be used to capture the cookie and then pass it to the access control logic that verifies permissions...

The drawback of this approach is that it requires updating all the endpoints that require **access control*

# Access control (2)

###  Step 2 - ... check if the user is allowed to execute the operation.


In methods that require access control, ***check if the proper*** user is calling the method.

```java
@Path("/ctrl")
public class ControlResource
{
	@Path("/version")
	@GET
	@Produces(MediaType.TEXT_HTML)
	public String version() {
        var session = Authentication.validateSession(ADMIN); // will throw NOT AUTHORIZED
        // user admin authenticated
    }
}
```        

Note: For the above to work, the method `Authentication.validateSession(ADMIN)` will need access to the cookie somehow...

# Access control (3)

It is possible to manage access control information separate from the web application logic

+ [ThreadLocal](https://www.baeldung.com/java-threadlocal) storage  - to store per invocation cookies;
+ [ContainerRequestFilter](https://jakarta.ee/specifications/platform/9.1/apidocs/?jakarta/ws/rs/container/ContainerRequestFilter.html) - to intercept REST invocations and manage cookies (and maybe other context)

### Store Cookies in ThreadLocal Storage

```java
public class RequestCookies {

    private static final ThreadLocal<Map<String, Cookie>> requestCookiesThreadLocal = new ThreadLocal<>();

    public static void set(Map<String, Cookie> cookies) {
    	requestCookiesThreadLocal.set(cookies);
    }

    public static  Map<String, Cookie> get() {
        return requestCookiesThreadLocal.get();
    }

    public static void clear() {
    	requestCookiesThreadLocal.remove();
    }
}
```

Thread local storage can serve as global variables but with a scope private to a particular thread. In this case,
we can use it to store the request cookies, which can then be accessed anywhere in the server code by the thread that is handling the request. Different requests will be handled by different threads, which will have their own private thread local storage.

The advantage is that the REST resource API does not have to be changed with annotations to capture the cookies (or to use other context invocation information).

### Intercept REST Requests

```java
@Provider
public class RequestCookiesFilter implements ContainerRequestFilter {

    @Override
    public void filter(ContainerRequestContext reqCtx) throws IOException {    	
    	RequestCookies.set( reqCtx.getCookies() );
    }
}
```

### Register the ContainerRequestFilter

```java
public MainApplication() {
	resources.add(ControlResource.class);		
	resources.add(RequestCookiesFilter.class);
    resources.add(Authentication.class);
}
```

Note: We register the filter like we do for normal REST resources.

## Access Control (4)

```java
static public Session validateSession(String userId) throws NotAuthorizedException {
	var cookies = RequestCookies.get();
	return validateSession( cookies.get(COOKIE_KEY ), userId );
}
	
static public Session validateSession(Cookie cookie, String userId) throws NotAuthorizedException {

	if (cookie == null )
		throw new NotAuthorizedException("No session initialized");
		
	var session = FakeRedisLayer.getInstance().getSession( cookie.getValue());
	if( session == null )
		throw new NotAuthorizedException("No valid session initialized");
			
	if (session.user() == null || session.user().length() == 0) 
		throw new NotAuthorizedException("No valid session initialized");
		
	if (!session.user().equals(userId))
		throw new NotAuthorizedException("Invalid user : " + session.user());
		
	return session;
}
```

# Access control (5)

###  Step 2 - ... check if the user is allowed to execute the operation.


In methods that require access control, ***check if the proper*** user is calling the method.

```java
@Path("/ctrl")
public class ControlResource
{
	@Path("/version")
	@GET
	@Produces(MediaType.TEXT_HTML)
	public String version() {
        var session = Authentication.validateSession(ADMIN); // will throw NOT AUTHORIZED
        // user admin authenticated
    }
}
```        

# Sample Code

The code provided [scc2425-lab9.zip](scc2425-lab9.zip) exemplify the techniques preseted above.


For testing it in the command line, just run:

```mvn clean compile package tomcat7:redeploy```

Try the control endpoints:

[http://127.0.0.1:8080/lab9/rest/ctrl/version](http://127.0.0.1:8080/lab9/rest/ctrl/version)

[http://127.0.0.1:8080/lab9/rest/ctrl/version2](http://127.0.0.1:8080/lab9/rest/ctrl/version2)

They should fail with `401 Status Code`, meaning Not Authorized

Open the login page:

[http://127.0.0.1:8080/lab9/rest/login](http://127.0.0.1:8080/lab9/rest/login)

Try `admin` and other usernames. Only `admin` should work.

Once `admin` is authenticated, both of the endpoints below should work.

[http://127.0.0.1:8080/lab9/rest/ctrl/version](http://127.0.0.1:8080/lab9/rest/ctrl/version)

[http://127.0.0.1:8080/lab9/rest/ctrl/version2](http://127.0.0.1:8080/lab9/rest/ctrl/version)

# TO DO


Add access control to Tukano Blob Storage. 

+ Only authenticated users should be able to upload and download blobs. 

+ Only authenticated 'admin' should be allowed to delete blobs.